<a href="https://colab.research.google.com/github/sergejsrk/SparqlDatasetAnalysis/blob/main/Covid19_datasets_03_July.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPARQL queries






In [ ]:
!pip install SPARQLWrapper

In [ ]:
import requests
#atlasam visas klases kopa ar tam piederoso instancu skaitu
sparql_query_all_classes="""
SELECT ?c (count(?x) as ?cx) where
{?x a ?c}
GROUP BY ?c
HAVING( count(?x) >= 1 )
ORDER BY desc(?cx)
#limit 10000
"""

r = requests.get( "https://covidontheweb.inria.fr/sparql",
                      params={'query':sparql_query_all_classes},
                      headers={'Accept': 'application/json'})
i=0
for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      # onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      # self.classes.append( onto_elem )
      i+=1
print(i)

sparql_query_class_connectivity="""
SELECT ?c (count(?x) as ?cx) where
{?x a ?c}
GROUP BY ?c
HAVING( count(?x) >= 1 )
ORDER BY desc(?cx)
#limit 10000
"""

r = requests.get( "https://covidontheweb.inria.fr/sparql",
                      params={'query':sparql_query_all_classes},
                      headers={'Accept': 'application/json'})
i=0
for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      # onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      # self.classes.append( onto_elem )
      i+=1
print(i)

7425
7425


In [ ]:
# Table output experiments

from IPython.display import HTML, display

def display_table(data):
    html = """<table border="1" >
              <thead>
              <tr>
                <th>URI</th>
                <th>Instance Count</th>
              </tr>
              </thead>
           """
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td width=\"80%%\">%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

data = [["really long class name  with a full uri on top ",2],["short name",6]]
display_table(data)

URI,Instance Count
really long class name with a full uri on top,2
short name,6


# Python classes for data collection

In [ ]:
import requests
import pickle

from IPython.display import HTML, display
from SPARQLWrapper import SPARQLWrapper, JSON

# url = "https://covidontheweb.inria.fr/sparql"
# sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"

# r = requests.get(url,
#                  params={'query':sparl_query_text},
#                  headers={'Accept': 'application/json'})
# print(r.content)
# print(f"Response: {r.json()}")
# res_json = r.json()

# print(res_json["results"]["bindings"])

# for e in res_json["results"]["bindings"]:
#   print(e["c"]["value"] + " " + e["cx"]["value"])

class DataSet:
  def __init__(self, name, endpoint_url):
    self.name = name
    self.endpoint_url = endpoint_url
    self.classes = []
    # self.properties = []

  def get_classes(self):
    sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"
    r = requests.get( self.endpoint_url,
                      params={'query':sparl_query_text},
                      headers={'Accept': 'application/json'},
                      verify=False)

    for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      self.classes.append( onto_elem )

  def print_classes(self, min_amount_of_instances, class_name):
    for c in self.classes:
      if len(class_name) == 0:
        print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" +
               c.short_name_fragment  + "\t" + c.namespace)
      else:
        if class_name in c.url and len(class_name) > 0 :
          print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" +
                c.short_name_fragment  + "\t" + c.namespace)
      if int(c.instance_count) < int(min_amount_of_instances):
        break

  def create_list_of_classes(self, min_amount_of_instances, class_name_substr):
    #if class_name_patern has len > 0 we show only classes whose name contains class_name_substr
    result = []
    for c in self.classes:
      if int(c.instance_count) < int(min_amount_of_instances):
        break

      if len(class_name_substr) == 0:
        result.append(c)
        # print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" +
        #        c.short_name_fragment  + "\t" + c.namespace)
      else:
        if class_name_substr in c.url and len(class_name_substr) > 0 :
          result.append(c)
          # print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" +
          #       c.short_name_fragment  + "\t" + c.namespace)
      if int(c.instance_count) < int(min_amount_of_instances):
        break

    return result

  def show_used_namespaces(self):
    namespace_map = {}
    for c in self.classes:
      if c.namespace in namespace_map:
        namespace_map[c.namespace] = namespace_map[c.namespace] + 1
      else:
        namespace_map[c.namespace] = 1
    count1 = 0
    for e in namespace_map:
      print(str(e) + "\t" + str(namespace_map[e]))
      count1 = count1 + namespace_map[e]

    print(count1)

  def is_class_present_in_ds(self, class_short_name):
    for c in self.classes:
      if class_short_name in c.url:
        return True

    return False

  def get_classes_from_ds(self, class_short_name):
    res = []
    for c in self.classes:
      if class_short_name in c.url:
        res.append(c)

    return res

  def collect_connectivity(self):
    print("Collecting connectivity ...")

    s = """
    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    """
    print(s)


    i=0
    for c in self.classes:
      print("Collecting connectivity for " + c.url + " " + str(i) + " / " + str(len(self.classes))  )
      classURI = c.url
      # sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"
      s = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
          ?expr ?p ?Drug.
          ?Drug a <"""+classURI+""">.
        }
      """
      # print(s)
      r = requests.get( self.endpoint_url,
                        params={'query':s},
                        headers={'Accept': 'application/json'},
                        verify=False)

      for e in r.json()["results"]["bindings"]:
          c.connectivity = e["expr_COUNT"]["value"]
          print(e["expr_COUNT"]["value"])
      i+=1

  def collect_en_class_labels(self):
    # specify the URI of the SPARQL endpoint to query
    sparql_endpoint = self.endpoint_url

    #session = requests.Session()
    #session.verify = False

    # create a SPARQLWrapper object and set the query string
    sparql = SPARQLWrapper(sparql_endpoint)
    #sparql.setHTTPSession(session)
    i = 0
    for c in self.classes:
      print(str(i) + " " + c.url)
      i=i+1
      sparql.setQuery("""
          SELECT ?label
          WHERE {
              <""" + c.url +"""> rdfs:label ?label .
              FILTER (langMatches(lang(?label), "en"))
          }
      """)

      # set the response format to JSON and execute the query
      sparql.setReturnFormat(JSON)
      # sparql.setVerify(False)
      results = sparql.query().convert()

      # process the results
      # print(results)
      for result in results["results"]["bindings"]:
          curr_label = result["label"]["value"]
          print("\t" + curr_label)
          if curr_label not in c.en_labels:
            c.en_labels.append(curr_label)


def show_class_list_as_html_table(class_list, ds_name, ds_url):
  html = f"""<table border="1" >
            <caption> List of classes in {ds_name} ({ds_url}) </caption>
            <thead>
            <tr>
              <th>URI</th>
              <th>Short Name</th>
              <th>En label</th>
              <th>Display name</th>
              <th>Instance Count</th>
              <th>Connectivity</th>
            </tr>
            </thead>
          """
  for c in class_list:
      html += "<tr>"
      html += "<td width=\"60%%\">%s</td>"%(c.url)
      html += "<td >%s</td>"%(c.short_name_fragment if len(c.short_name_fragment) > 0 else c.short_name )
      en_labels_str = ""
      for en_l in c.en_labels:
        en_labels_str += en_l + "\n"
      html += "<td >%s</td>"%(en_labels_str)
      html += "<td >%s</td>"%(c.get_display_name())
      html += "<td >%s</td>"%(c.instance_count)
      html += "<td >%s</td>"%(c.connectivity)
      html += "</tr>"
  html += "</table>"
  display(HTML(html))


class OntologyElement:
  def __init__(self, url, instance_count):
    self.url = url
    self.instance_count = instance_count
    self.short_name = ""
    self.short_name_fragment = ""
    self.prefix = ""
    self.namespace = ""
    self.connectivity = 0
    self.en_label = ""
    self.en_labels = []


    if self.url.rfind("/") != -1:
      self.short_name = self.url[self.url.rfind("/") + 1:]
      self.namespace = self.url[:self.url.rfind("/") ]
    if self.short_name.find("#") != -1:
      self.short_name_fragment = self.short_name[self.short_name.find("#") + 1:]
      self.namespace = self.url[:self.url.rfind("#") ]

    # self.short_name = self.url[self.url.rfind("/"):]
    # self.short_name_fragment = self.short_name[self.short_name.find("#"):]


  def get_display_name(self):
    # yago klasēm atmest tikai numuru, ja tam ir vismaz 6 cipari (lai
    # neatmestu gadus)
    if "yago" in self.url:
      #ja yago klase beidzas ar vairak neka 6 cipariem - nogriezam visus ciparus, kas atrodas URL beigas
      if len(self.short_name) > 7:
        if self.short_name[-6:].isdigit():
          res = self.short_name.rstrip('0123456789')
          return res
      return self.short_name
    # wikidata Q_nnn klasēm vienmēr skatīties anglisko label
    elif "www.wikidata.org/entity/Q" in self.url:
      if len(self.en_labels) > 0:
        # nemam pirmo labeli
        return self.en_labels[0]
    else:
      # citās situācijās novērtējam, lai label ir būtiski vairāk tekstuālas
      # informācijas (vairāk burtu)
      lettersInUrl =  len([c for c in (self.short_name_fragment if len(self.short_name_fragment) > 0 else self.short_name) if c.isalpha()])
      lettersInLabel = 0
      if len(self.en_labels) > 0:
        lettersInLabel = len([c for c in self.en_labels[0] if c.isalpha()])
      if lettersInLabel > lettersInUrl:
        return self.en_labels[0]
      else:
        return (self.short_name_fragment if len(self.short_name_fragment) > 0 else self.short_name)

    return ""






# Executing code for data collection from several endpoints


## covid19ontheweb

In [ ]:
ds1 = DataSet("covid19ontheweb", "https://covidontheweb.inria.fr/sparql")
ds1.get_classes()
ds1.collect_en_class_labels()

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covidontheweb.inria.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


0 http://www.w3.org/ns/oa#Annotation
1 http://www.w3.org/ns/oa#TextQuoteSelector
2 http://www.w3.org/ns/prov#Entity
3 http://www.w3.org/ns/oa#TextPositionSelector
4 http://ns.inria.fr/wasabi/ontology/Song
5 http://purl.org/spar/fabio/Work
	work
	work
	work
6 http://purl.org/spar/fabio/JournalArticle
	journal article
	journal article
	journal article
7 http://schema.org/Observation
8 http://ns.inria.fr/wasabi/ontology/Album
9 http://www.openkg.cn/COVID-19/research/class/C1
	Virus
	Virus
10 http://schema.org/SpecialAnnouncement
11 http://xmlns.com/foaf/0.1/Person
12 http://www.arg.dundee.ac.uk/aif#I-node
13 http://schema.org/ScholarlyArticle
14 http://purl.org/ontology/bibo/AcademicArticle
	Academic Article
15 http://purl.org/spar/fabio/Abstract
	abstract
	abstract
	abstract
16 http://ns.inria.fr/covid19/DocumentBody
17 http://purl.org/spar/fabio/ResearchPaper
	research paper
	research paper
	research paper
18 http://www.arg.dundee.ac.uk/aif#KnowledgePosition_Statement
19 http://purl.org

In [ ]:
ds1 = DataSet("covid19ontheweb", "https://covidontheweb.inria.fr/sparql")
ds1.get_classes()
ds1.collect_connectivity()
ds1.collect_en_class_labels()
#ds1.show_used_namespaces()
# print()
#ds1.print_classes(10,"")
#show_class_list_as_html_table( ds1.create_list_of_classes(10,""), ds1.name, ds1.endpoint_url )

with open('ds1.pickle', 'wb') as f:
    pickle.dump(ds1, f)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covidontheweb.inria.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covidontheweb.inria.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


2


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covidontheweb.inria.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

262


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covidontheweb.inria.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


3273


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covidontheweb.inria.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


655
0 http://www.w3.org/ns/oa#Annotation
1 http://www.w3.org/ns/oa#TextQuoteSelector
2 http://www.w3.org/ns/prov#Entity
3 http://www.w3.org/ns/oa#TextPositionSelector
4 http://ns.inria.fr/wasabi/ontology/Song
5 http://purl.org/spar/fabio/Work
	work
	work
	work
6 http://purl.org/spar/fabio/JournalArticle
	journal article
	journal article
	journal article
7 http://schema.org/Observation
8 http://ns.inria.fr/wasabi/ontology/Album
9 http://www.openkg.cn/COVID-19/research/class/C1
	Virus
	Virus
10 http://schema.org/SpecialAnnouncement
11 http://xmlns.com/foaf/0.1/Person
12 http://www.arg.dundee.ac.uk/aif#I-node
13 http://schema.org/ScholarlyArticle
14 http://purl.org/ontology/bibo/AcademicArticle
	Academic Article
15 http://purl.org/spar/fabio/Abstract
	abstract
	abstract
	abstract
16 http://ns.inria.fr/covid19/DocumentBody
17 http://purl.org/spar/fabio/ResearchPaper
	research paper
	research paper
	research paper
18 http://www.arg.dundee.ac.uk/aif#KnowledgePosition_Statement
19 http://purl

In [ ]:
print()
#ds1.print_classes(10,"")
with open("ds1.pickle", "rb") as f:
    # Load the pickled data into a Python object
    ds1 = pickle.load(f)
show_class_list_as_html_table( ds1.create_list_of_classes(10,""), ds1.name, ds1.endpoint_url )

URI,Short Name,En label,Display name,Instance Count,Connectivity
http://www.w3.org/ns/oa#Annotation,Annotation,,Annotation,74796551,2
http://www.w3.org/ns/oa#TextQuoteSelector,TextQuoteSelector,,TextQuoteSelector,74796550,74796550
http://www.w3.org/ns/prov#Entity,Entity,,Entity,74280968,12
http://www.w3.org/ns/oa#TextPositionSelector,TextPositionSelector,,TextPositionSelector,74280960,74280960
http://ns.inria.fr/wasabi/ontology/Song,Song,,Song,4188780,2
http://purl.org/spar/fabio/Work,Work,work,Work,1226449,1804914
http://purl.org/spar/fabio/JournalArticle,JournalArticle,journal article,JournalArticle,1124956,1658498
http://schema.org/Observation,Observation,,Observation,658177,658177
http://ns.inria.fr/wasabi/ontology/Album,Album,,Album,208743,2099284
http://www.openkg.cn/COVID-19/research/class/C1,C1,Virus,Virus,199494,4


## tweetscov19

In [ ]:
ds2 = DataSet("tweetscov19", "https://data.gesis.org/tweetscov19/sparql")
ds2.get_classes()
# ds2.collect_connectivity()
ds2.collect_en_class_labels()
# ds2.show_used_namespaces()

with open('ds2.pickle', 'wb') as f:
    pickle.dump(ds2, f)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.gesis.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


0 http://schema.org/InteractionCounter
1 http://rdfs.org/sioc/ns#UserAccount
2 http://www.ics.forth.gr/isl/oae/core#Entity
	entity
3 http://rdfs.org/sioc/ns#Post
4 http://www.gsi.dit.upm.es/ontologies/onyx/ns#EmotionSet
5 http://rdfs.org/sioc/types#Tag
	Tag
6 http://dbpedia.org/class/yago/Colleague109935990
7 http://dbpedia.org/class/yago/Engineer109615807
8 http://dbpedia.org/class/yago/Scientist110560637
9 http://dbpedia.org/class/yago/Inventor110214637
10 http://dbpedia.org/class/yago/Donor110025730
11 http://dbpedia.org/class/yago/Philanthropist110421956
12 http://dbpedia.org/class/yago/ComputerUser109951274
13 http://dbpedia.org/class/yago/WikicatOfficersOfTheOrderOfTheBritishEmpire
14 http://dbpedia.org/class/yago/System108435388
15 http://dbpedia.org/class/yago/ComputerScientist109951070
16 http://dbpedia.org/class/yago/WikicatAmericanPhilanthropists
17 http://dbpedia.org/class/yago/WikicatAmericanInventors
18 http://dbpedia.org/class/yago/WikicatAmericanComputerScientists
19 ht

In [ ]:
print()
# ds2.print_classes(10,"")
show_class_list_as_html_table( ds2.create_list_of_classes(2,""), ds2.name, ds2.endpoint_url )

URI,Short Name,En label,Display name,Instance Count,Connectivity
http://schema.org/InteractionCounter,InteractionCounter,,InteractionCounter,40525292,0
http://rdfs.org/sioc/ns#UserAccount,UserAccount,,UserAccount,32477877,0
http://www.ics.forth.gr/isl/oae/core#Entity,Entity,entity,Entity,27570278,0
http://rdfs.org/sioc/ns#Post,Post,,Post,20262647,0
http://www.gsi.dit.upm.es/ontologies/onyx/ns#EmotionSet,EmotionSet,,EmotionSet,20262646,0
http://rdfs.org/sioc/types#Tag,Tag,Tag,Tag,13804256,0
http://dbpedia.org/class/yago/Colleague109935990,Colleague109935990,,Colleague,9999,0
http://dbpedia.org/class/yago/Engineer109615807,Engineer109615807,,Engineer,9999,0
http://dbpedia.org/class/yago/Scientist110560637,Scientist110560637,,Scientist,9999,0
http://dbpedia.org/class/yago/Inventor110214637,Inventor110214637,,Inventor,5625,0


## SIB COVID 19

In [ ]:
ds3 = DataSet("SIB COVID 19", "https://covid-19-sparql.expasy.org/sparql")
ds3.get_classes()
ds3.collect_connectivity()
ds3.collect_en_class_labels()
# ds3.show_used_namespaces()

with open('ds3.pickle', 'wb') as f:
    pickle.dump(ds3, f)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid-19-sparql.expasy.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid-19-sparql.expasy.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


10491650


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid-19-sparql.expasy.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid-19-sparql.expasy.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


2


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid-19-sparql.expasy.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


10
0 http://biohackathon.org/resource/faldo#ExactPosition
1 http://biohackathon.org/resource/faldo#Region
2 http://biohackathon.org/resource/faldo#Position
3 http://biohackathon.org/resource/vg#Step
4 http://biohackathon.org/resource/faldo#ForwardStrandPosition
5 http://ddbj.nig.ac.jp/ontologies/nucleotide/Exon
6 http://ddbj.nig.ac.jp/ontologies/nucleotide/Mature_Peptide
7 http://ddbj.nig.ac.jp/ontologies/nucleotide/Coding_Sequence
8 http://ddbj.nig.ac.jp/ontologies/nucleotide/Protein
9 http://ddbj.nig.ac.jp/ontologies/nucleotide/Gene
10 http://nextprot.org/rdf#Xref
11 http://biohackathon.org/resource/vg#Node
12 http://purl.org/genex#Expression
	gene expression
13 http://ddbj.nig.ac.jp/ontologies/nucleotide/Stem_Loop
14 http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement
15 http://nextprot.org/rdf#Evidence
16 http://purl.obolibrary.org/obo/BFO_0000001
	entity
17 http://purl.obolibrary.org/obo/BFO_0000002
	continuant
18 http://purl.org/genex#ExpressionCondition
	gene expression experim

In [ ]:
print()
# ds3.print_classes(10,"")
show_class_list_as_html_table( ds3.create_list_of_classes(10,""), ds3.name, ds3.endpoint_url )

URI,Short Name,En label,Display name,Instance Count,Connectivity
http://biohackathon.org/resource/faldo#ExactPosition,ExactPosition,,ExactPosition,5281316,10491650
http://biohackathon.org/resource/faldo#Region,Region,,Region,5245908,108449
http://biohackathon.org/resource/faldo#Position,Position,,Position,5204334,10403218
http://biohackathon.org/resource/vg#Step,Step,,Step,5198168,0
http://biohackathon.org/resource/faldo#ForwardStrandPosition,ForwardStrandPosition,,ForwardStrandPosition,77084,88598
http://ddbj.nig.ac.jp/ontologies/nucleotide/Exon,Exon,,Exon,39936,124238
http://ddbj.nig.ac.jp/ontologies/nucleotide/Mature_Peptide,Mature_Peptide,,Mature_Peptide,25161,0
http://ddbj.nig.ac.jp/ontologies/nucleotide/Coding_Sequence,Coding_Sequence,,Coding_Sequence,15820,0
http://ddbj.nig.ac.jp/ontologies/nucleotide/Protein,Protein,,Protein,15820,15820
http://ddbj.nig.ac.jp/ontologies/nucleotide/Gene,Gene,,Gene,14505,90402


## Covid 19 KG

In [ ]:
ds4 = DataSet("Covid 19 KG", "https://sparql.proconsortium.org/virtuoso5/sparql")
ds4.get_classes()
ds4.collect_connectivity()
ds4.collect_en_class_labels()
# ds4.show_used_namespaces()

with open('ds4.pickle', 'wb') as f:
    pickle.dump(ds4, f)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sparql.proconsortium.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



    Connectiviy is collected by sendidng following requests for each class in the given dataset
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <classURI>.
      }
    


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sparql.proconsortium.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


38265866


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sparql.proconsortium.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

2


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sparql.proconsortium.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sparql.proconsortium.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


9
0 https://semrep.nlm.nih.gov/Entity
	Entity
	Entity
1 http://www.w3.org/2002/07/owl#Axiom
2 https://semrep.nlm.nih.gov/Relation
	Relation
	Relation
3 http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement
4 http://biohackathon.org/resource/faldo#ExactPosition
5 http://biohackathon.org/resource/faldo#Position
6 http://biohackathon.org/resource/faldo#Region
7 http://purl.uniprot.org/core/Domain_Extent_Annotation
8 http://purl.uniprot.org/core/Transmembrane_Annotation
9 http://purl.uniprot.org/core/Nucleotide_Resource
10 http://purl.uniprot.org/core/Strain
11 http://www.w3.org/2002/07/owl#Restriction
12 http://www.w3.org/2002/07/owl#Class
13 http://purl.uniprot.org/core/Structured_Name
14 http://purl.uniprot.org/core/Catalytic_Activity
15 http://purl.uniprot.org/core/Catalytic_Activity_Annotation
16 http://purl.uniprot.org/core/Region_Annotation
17 http://purl.uniprot.org/core/Citation_Statement
18 http://purl.uniprot.org/core/Function_Annotation
19 http://purl.uniprot.org/core/Compositio

In [ ]:
print()
# ds4.print_classes(10,"")
show_class_list_as_html_table( ds4.create_list_of_classes(10,""), ds4.name, ds4.endpoint_url )

URI,Short Name,En label,Display name,Instance Count,Connectivity
https://semrep.nlm.nih.gov/Entity,Entity,Entity,Entity,38265866,38265866
http://www.w3.org/2002/07/owl#Axiom,Axiom,,Axiom,11938498,0
https://semrep.nlm.nih.gov/Relation,Relation,Relation,Relation,2173171,2173171
http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement,Statement,,Statement,394660,36622
http://biohackathon.org/resource/faldo#ExactPosition,ExactPosition,,ExactPosition,240757,261782
http://biohackathon.org/resource/faldo#Position,Position,,Position,240757,261782
http://biohackathon.org/resource/faldo#Region,Region,,Region,130891,130939
http://purl.uniprot.org/core/Domain_Extent_Annotation,Domain_Extent_Annotation,,Domain_Extent_Annotation,49386,98772
http://purl.uniprot.org/core/Transmembrane_Annotation,Transmembrane_Annotation,,Transmembrane_Annotation,45146,90292
http://purl.uniprot.org/core/Nucleotide_Resource,Nucleotide_Resource,,Nucleotide_Resource,35426,68967


# Checking for presence of preknown interesting classes in datasets

In [ ]:

#Interesting classes from Covid on the web
interesting_classes = ["Person",
                       "Place",
                       "Agent",
                       "Chemical", #"Chemical Substance"
                       "Disease",
                       "Procedure",
                       "Intervention",
                       "Claim",
                       "Outcome",
                       "Viruses",
                       "Biomolecule",
                       "Symptom",
                       "Chemical",
                       "Drugs",
                       "Mutation",
                       "Species",
                       "Device",
                       "Stain",
                       "Medication",
                       "Condition",
                       "Gene",
                       "Protein",
                       "Work",
                       "Position",
                       "Evidence"
                       ]

print("Searching for interesting classes in " + ds1.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds1.print_classes(1, c)
  print()



Searching for interesting classes in https://covidontheweb.inria.fr/sparql

Searching for Person
http://xmlns.com/foaf/0.1/Person	135614	Person		http://xmlns.com/foaf/0.1
http://ns.inria.fr/wasabi/ontology/Artist_Person	24264	Artist_Person		http://ns.inria.fr/wasabi/ontology
http://dbpedia.org/ontology/Person	5608	Person		http://dbpedia.org/ontology
http://dbpedia.org/resource/Person	203	Person		http://dbpedia.org/resource
http://dbpedia.org/ontology/MilitaryPerson	73	MilitaryPerson		http://dbpedia.org/ontology

Searching for Place
http://dbpedia.org/ontology/Place	6810	Place		http://dbpedia.org/ontology
http://dbpedia.org/ontology/PopulatedPlace	3835	PopulatedPlace		http://dbpedia.org/ontology
http://dbpedia.org/ontology/NaturalPlace	358	NaturalPlace		http://dbpedia.org/ontology
http://dbpedia.org/resource/Place	43	Place		http://dbpedia.org/resource
http://dbpedia.org/ontology/HistoricPlace	37	HistoricPlace		http://dbpedia.org/ontology

Searching for Agent
http://dbpedia.org/ontology/

In [ ]:
# interesting classes from Tweets KG
interesting_classes = ["Person", "Website", "Post", "UserAccount", "Disease", "ChemicalSubstance", "Agent", "Place" ]

print("Searching for interesting classes in " + ds2.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds2.print_classes(1, c)
  print()


Searching for interesting classes in https://data.gesis.org/tweetscov19/sparql

Searching for Person
http://dbpedia.org/class/yago/Person100007846	18	Person100007846		http://dbpedia.org/class/yago
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson	15	DUL.owl#NaturalPerson	NaturalPerson	http://www.ontologydesignpatterns.org/ont/dul/DUL.owl
http://schema.org/Person	15	Person		http://schema.org
http://xmlns.com/foaf/0.1/Person	15	Person		http://xmlns.com/foaf/0.1
http://dbpedia.org/ontology/Person	15	Person		http://dbpedia.org/ontology
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#SocialPerson	13	DUL.owl#SocialPerson	SocialPerson	http://www.ontologydesignpatterns.org/ont/dul/DUL.owl
http://dbpedia.org/ontology/PersonFunction	8	PersonFunction		http://dbpedia.org/ontology
http://umbel.org/umbel/rc/PersonWithOccupation	6	PersonWithOccupation		http://umbel.org/umbel/rc
http://dbpedia.org/class/yago/ReligiousPerson109628382	2	ReligiousPerson109628382		http://dbpedia.or

In [ ]:

# interesting classes from SIB Covid 19
interesting_classes = ["Gene", "Protein", "Work", "Position", "Evidence" ]

print("Searching for interesting classes in " + ds3.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds4.print_classes(1, c)
  print()

Searching for interesting classes in https://covid-19-sparql.expasy.org/sparql

Searching for Gene
http://purl.uniprot.org/core/Gene	7031	Gene		http://purl.uniprot.org/core

Searching for Protein
http://purl.uniprot.org/core/Protein	7041	Protein		http://purl.uniprot.org/core

Searching for Work

Searching for Position
http://biohackathon.org/resource/faldo#ExactPosition	240757	faldo#ExactPosition	ExactPosition	http://biohackathon.org/resource/faldo
http://biohackathon.org/resource/faldo#Position	240757	faldo#Position	Position	http://biohackathon.org/resource/faldo

Searching for Evidence



In [ ]:
# interesting classes from Covid 19  KG
interesting_classes = ["Stain", "Strain", "Medication", "Condition", "Gene", "Protein", "Work", "Position" ]

print("Searching for interesting classes in " + ds4.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds4.print_classes(1, c)
  print()

Searching for interesting classes in https://sparql.proconsortium.org/virtuoso5/sparql

Searching for Stain

Searching for Strain
http://purl.uniprot.org/core/Strain	33541	Strain		http://purl.uniprot.org/core

Searching for Medication

Searching for Condition

Searching for Gene
http://purl.uniprot.org/core/Gene	7031	Gene		http://purl.uniprot.org/core

Searching for Protein
http://purl.uniprot.org/core/Protein	7041	Protein		http://purl.uniprot.org/core

Searching for Work

Searching for Position
http://biohackathon.org/resource/faldo#ExactPosition	240757	faldo#ExactPosition	ExactPosition	http://biohackathon.org/resource/faldo
http://biohackathon.org/resource/faldo#Position	240757	faldo#Position	Position	http://biohackathon.org/resource/faldo



# Class presence in multiple datasets as table

In [ ]:
# Classes that are present in more than one data set

#obtain list of all classes - take short class name not full url

all_class_names = set()

datasets = [ds1, ds2, ds3, ds4]

for ds in datasets:
  for c in ds.classes:
    if len(c.short_name_fragment) > 0:
      all_class_names.add( c.short_name_fragment )
    else:
      all_class_names.add( c.short_name )

# print(all_class_names)

# check if this name is present in each of the datasets
# name is short name
name_2_presence_in_ds = {}
for name in all_class_names:
  # print (name)
  presence = [[]] * len(datasets)
  # print(presence)
  for i in range(0,len(datasets)):
    ds = datasets[i]
    list_of_cl = ds.get_classes_from_ds(name)
    if len(list_of_cl) > 0:
      presence[i] = [(cl.short_name, cl.instance_count) for cl in list_of_cl]

  # print(name + " " +str(presence))
  name_2_presence_in_ds[name] = presence

print(len(name_2_presence_in_ds))

print(name_2_presence_in_ds)




def show_class_presence_in_ds_as_html(data, datasets):
  html = f"""<table border="1" >
            <caption> Class presence in datasets </caption>
            <thead>
            <tr>
              <th>Short Class Name</th>
        """
  for ds in datasets:
    html += f"<th>{ds.name}</th>\n"

  html += """
            </tr>
            </thead>
          """
  for name in data:
      html += "<tr>"
      html += "<td >%s</td>"%(name)
      x = data[name]
      # print(x)
      i=0
      for ds in datasets:
        if len(x[i]) > 0:
          html += "<td >"
          for e in x[i]:
            html+=e[0] + " : " + e[1] + "\n<br/>"
          html += "</td>"
        else:
          html += f"<td ></td>"
        i+=1

      html += "</tr>"
  html += "</table>"
  display(HTML(html))
  print(data)


show_class_presence_in_ds_as_html(name_2_presence_in_ds, datasets)


1933
{'Q152': [[('Q15265344', '231'), ('Q152', '115'), ('Q15275719', '34')], [('Q15265344', '5')], [], []], 'WikicatWoodworkers': [[], [('WikicatWoodworkers', '1')], [], []], 'Information106634376': [[], [('Information106634376', '1')], [], []], 'Concept105835747': [[], [('Concept105835747', '2')], [], []], 'Q46970': [[('Q46970', '45')], [], [], []], 'Phenomenon': [[('Phenomenon', '90')], [], [], []], 'WikicatAmericanPeopleOfSicilianDescent': [[], [('WikicatAmericanPeopleOfSicilianDescent', '1')], [], []], 'Q212238': [[('Q212238', '49')], [], [], []], 'Q614112': [[('Q614112', '106')], [], [], []], 'JournalArticle': [[('JournalArticle', '1124956')], [], [], []], 'Instrumentality103575240': [[], [('Instrumentality103575240', '6')], [], []], 'GoBiologicalProcess': [[], [], [('rdf#GoBiologicalProcess', '42')], []], 'Q6256': [[('Q6256', '357')], [('Q6256', '3')], [], []], 'Muscle': [[('Muscle', '40')], [], [], []], 'WikicatPharmaceuticalCompanies': [[], [('WikicatPharmaceuticalCompanies', '

Short Class Name,covid19ontheweb,tweetscov19,SIB COVID 19,Covid 19 KG
Q152,Q15265344 : 231 Q152 : 115 Q15275719 : 34,Q15265344 : 5,,
WikicatWoodworkers,,WikicatWoodworkers : 1,,
Information106634376,,Information106634376 : 1,,
Concept105835747,,Concept105835747 : 2,,
Q46970,Q46970 : 45,,,
Phenomenon,Phenomenon : 90,,,
WikicatAmericanPeopleOfSicilianDescent,,WikicatAmericanPeopleOfSicilianDescent : 1,,
Q212238,Q212238 : 49,,,
Q614112,Q614112 : 106,,,
JournalArticle,JournalArticle : 1124956,,,


{'Q152': [[('Q15265344', '231'), ('Q152', '115'), ('Q15275719', '34')], [('Q15265344', '5')], [], []], 'WikicatWoodworkers': [[], [('WikicatWoodworkers', '1')], [], []], 'Information106634376': [[], [('Information106634376', '1')], [], []], 'Concept105835747': [[], [('Concept105835747', '2')], [], []], 'Q46970': [[('Q46970', '45')], [], [], []], 'Phenomenon': [[('Phenomenon', '90')], [], [], []], 'WikicatAmericanPeopleOfSicilianDescent': [[], [('WikicatAmericanPeopleOfSicilianDescent', '1')], [], []], 'Q212238': [[('Q212238', '49')], [], [], []], 'Q614112': [[('Q614112', '106')], [], [], []], 'JournalArticle': [[('JournalArticle', '1124956')], [], [], []], 'Instrumentality103575240': [[], [('Instrumentality103575240', '6')], [], []], 'GoBiologicalProcess': [[], [], [('rdf#GoBiologicalProcess', '42')], []], 'Q6256': [[('Q6256', '357')], [('Q6256', '3')], [], []], 'Muscle': [[('Muscle', '40')], [], [], []], 'WikicatPharmaceuticalCompanies': [[], [('WikicatPharmaceuticalCompanies', '1')],

# Printing class names after sort by presence in datasets

In [ ]:
sorted_by_presence = []
short_name_2_presence_and_instance_count = []
for short_name in name_2_presence_in_ds:
  # print(short_name)
  #number of datasets containing clases matching this short name
  presence_count = 0
  curr_presence = name_2_presence_in_ds[short_name]
  # print("\t" + str(curr_presence))
  inst_count_list = [0]*4
  e_ind = 0
  for e in curr_presence:
    # print(e)
    if len(e) > 0:
      presence_count += 1

      z = 0
      for i in e:
        z += int(i[1])
      inst_count_list[e_ind]=z
    e_ind += 1

  short_name_2_presence_and_instance_count.append(
      (
          short_name, presence_count,
          inst_count_list[0] if len(inst_count_list)>0 else 0,
          inst_count_list[1] ,
          inst_count_list[2] ,
          inst_count_list[3] ,
       )
  )

# print(short_name_2_presence_and_instance_count)

for i in sorted(short_name_2_presence_and_instance_count, reverse = True, key=lambda x: (x[1])):
  # if i[2] > 1 and i[3]>1 and i[4]>1 and i[5]>1:
  # if (i[2] + i[3] + i[4] + i[5]) > 1000 and "Person" in i[0]:
  if (i[2] + i[3] + i[4] + i[5]) > 10 and  i[1]>=3:
    print(i)
    # print(name_2_presence_in_ds[i[0]])
    # print()



# max_score = 0
# obj_with_max_score = None
# for e in name_2_presence_in_ds:
#   print(e + " " + str(name_2_presence_in_ds[e]))
#   curr_score = 0
#   for i in range(0, len(datasets)):
#     if name_2_presence_in_ds[e][i]:
#       curr_score += 1
#   if curr_score > max_score:
#     max_score = curr_score
#     obj_with_max_score =


('Format', 4, 318, 230, 228, 228)
('Part', 4, 726, 22, 509, 3758)
('array-of-QuadMap', 4, 98, 111, 111, 111)
('Statement', 4, 62609, 3, 6227, 403435)
('Function', 4, 5002, 8, 172, 8120)
('Site', 4, 49, 10, 147, 2354)
('AnnotationProperty', 4, 301, 5, 23, 5)
('Gene', 4, 90, 1, 15021, 7031)
('Ontology', 4, 41, 8, 4, 7)
('Act', 4, 554, 88, 102, 22768)
('Property', 4, 34153, 40, 133, 235)
('array-of-QuadMapFormat', 4, 98, 98, 98, 98)
('Location', 4, 32, 24, 102, 6484)
('Form', 4, 880, 238, 228, 228)
('Entity', 4, 74281000, 27570358, 735, 38265867)
('State', 4, 63094, 74, 6227, 403435)
('Position', 4, 74343435, 2, 10562936, 481514)
('Name', 4, 851, 5, 483, 15757)
('Annotation', 4, 74796852, 5, 3382, 168168)
('Domain', 4, 34, 1, 187, 51988)
('Service', 4, 79, 5, 1, 7)
('Region', 4, 983, 21, 5245972, 140429)
('S', 4, 153645238, 20282252, 5306573, 477907)
('Class', 4, 19218, 59, 134, 19691)
('Structure', 4, 3433, 1, 796, 17280)
('Activity', 4, 229, 3, 68, 22744)
('QuadMap', 4, 228, 265, 265, 2

# Downloading dataset files from github or preparing to use local dataset files (if present)

In [ ]:
from __future__ import print_function
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import pickle

url = 'https://github.com/sergejsrk/SparqlDatasetAnalysis/raw/main/'

last_percent_reported = None
data_root = '.' # Change me to store data elsewhere

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()

    last_percent_reported = percent

def maybe_download(filename, expected_bytes=0, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename)
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  else:
    print('File found on disk. Using existing file ', filename)
  # statinfo = os.stat(dest_filename)
  # if statinfo.st_size == expected_bytes:
  #   print('Found and verified', dest_filename)
  # else:
  #   raise Exception(
  #     'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

ds1_filename = maybe_download('ds1.pickle')
ds2_filename = maybe_download('ds2.pickle')
ds3_filename = maybe_download('ds3.pickle')
ds4_filename = maybe_download('ds4.pickle')

# test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)


file = open("ds1.pickle",'rb')
restoredDs1 = pickle.load(file)
file.close()

file = open("ds2.pickle",'rb')
restoredDs2 = pickle.load(file)
file.close()

file = open("ds3.pickle",'rb')
restoredDs3 = pickle.load(file)
file.close()

file = open("ds4.pickle",'rb')
restoredDs4 = pickle.load(file)
file.close()

# print(restoredDs1)
# print(restoredDs1.classes)
# show_class_list_as_html_table( restoredDs1.create_list_of_classes(10,""), restoredDs1.name, restoredDs1.endpoint_url )



File found on disk. Using existing file  ds1.pickle
File found on disk. Using existing file  ds2.pickle
File found on disk. Using existing file  ds3.pickle
File found on disk. Using existing file  ds4.pickle


# Presenting dataset info in a sortable and filterable way


In [ ]:
from google.colab import data_table

data_table.enable_dataframe_formatter()

import numpy as np
import pandas as pd

def create_dataframe_from_dataset(dataset_prm):
  data = []
  for c in dataset_prm.classes:
    data.append([ c.url,
               c.short_name_fragment if len(c.short_name_fragment) > 0 else c.short_name ,
               int(c.instance_count),
               int(c.connectivity)])

  df = pd.DataFrame(
    data,
	  # index = [0,1,2],
	  columns = ['URI','Short Name', 'Instance Count', 'Connectivity']
  )

  return df


## covid19ontheweb

In [ ]:
print(f"Showing data for dataset {restoredDs1.name} ({restoredDs1.endpoint_url})")
create_dataframe_from_dataset(restoredDs1)

Showing data for dataset covid19ontheweb (https://covidontheweb.inria.fr/sparql)


,URI,Short Name,Instance Count,Connectivity
0,http://www.w3.org/ns/oa#Annotation,Annotation,74796551,2
1,http://www.w3.org/ns/oa#TextQuoteSelector,TextQuoteSelector,74796550,74796550
2,http://www.w3.org/ns/prov#Entity,Entity,74280968,12
3,http://www.w3.org/ns/oa#TextPositionSelector,TextPositionSelector,74280960,74280960
4,http://ns.inria.fr/wasabi/ontology/Song,Song,4188780,2
...,...,...,...,...
995,http://dbpedia.org/resource/Devices,Devices,22,753
996,http://dbpedia.org/resource/Behavior,Behavior,22,928
997,http://dbpedia.org/resource/Right,Right,22,262
998,http://dbpedia.org/resource/Tendency,Tendency,22,3273


## tweetscov19

In [ ]:
print(f"Showing data for dataset {restoredDs2.name} ({restoredDs2.endpoint_url})")
create_dataframe_from_dataset(restoredDs2)

Showing data for dataset tweetscov19 (https://data.gesis.org/tweetscov19/sparql)


,URI,Short Name,Instance Count,Connectivity
0,http://schema.org/InteractionCounter,InteractionCounter,40525292,0
1,http://rdfs.org/sioc/ns#UserAccount,UserAccount,32477877,0
2,http://www.ics.forth.gr/isl/oae/core#Entity,Entity,27570278,0
3,http://rdfs.org/sioc/ns#Post,Post,20262647,0
4,http://www.gsi.dit.upm.es/ontologies/onyx/ns#E...,EmotionSet,20262646,0
...,...,...,...,...
915,http://dbpedia.org/class/yago/WikicatHumanRights,WikicatHumanRights,1,0
916,http://dbpedia.org/class/yago/Writing106362953,Writing106362953,1,0
917,http://dbpedia.org/class/yago/Sensualist109629246,Sensualist109629246,1,0
918,http://dbpedia.org/class/yago/WikicatBisexualP...,WikicatBisexualPeople,1,0


## SIB COVID 19

In [ ]:
print(f"Showing data for dataset {restoredDs3.name} ({restoredDs3.endpoint_url})")
create_dataframe_from_dataset(restoredDs3)

Showing data for dataset SIB COVID 19 (https://covid-19-sparql.expasy.org/sparql)


,URI,Short Name,Instance Count,Connectivity
0,http://biohackathon.org/resource/faldo#ExactPo...,ExactPosition,5281316,10491650
1,http://biohackathon.org/resource/faldo#Region,Region,5245908,108449
2,http://biohackathon.org/resource/faldo#Position,Position,5204334,10403218
3,http://biohackathon.org/resource/vg#Step,Step,5198168,0
4,http://biohackathon.org/resource/faldo#Forward...,ForwardStrandPosition,77084,88598
...,...,...,...,...
216,http://purl.uniprot.org/core/Rank,Rank,1,17
217,http://purl.uniprot.org/core/Patent_Citation,Patent_Citation,1,3
218,http://www.w3.org/ns/sparql-service-descriptio...,Dataset,1,1
219,http://purl.uniprot.org/core/Erroneous_Initiat...,Erroneous_Initiation_Annotation,1,2


## Covid 19 KG

In [ ]:
print(f"Showing data for dataset {restoredDs4.name} ({restoredDs4.endpoint_url})")
create_dataframe_from_dataset(restoredDs4)

Showing data for dataset Covid 19 KG (https://sparql.proconsortium.org/virtuoso5/sparql)


,URI,Short Name,Instance Count,Connectivity
0,https://semrep.nlm.nih.gov/Entity,Entity,38265866,38265866
1,http://www.w3.org/2002/07/owl#Axiom,Axiom,11938498,0
2,https://semrep.nlm.nih.gov/Relation,Relation,2173171,2173171
3,http://www.w3.org/1999/02/22-rdf-syntax-ns#Sta...,Statement,394660,36622
4,http://biohackathon.org/resource/faldo#ExactPo...,ExactPosition,240757,261782
...,...,...,...,...
107,http://purl.org/goodrelations/v1#ActualProduct...,ActualProductOrServicesInstance,1,5
108,http://www.ebusiness-unibw.org/ontologies/ecla...,C_AKE112003-tax,1,1
109,http://purl.uniprot.org/core/NP_Binding_Annota...,NP_Binding_Annotation,1,2
110,http://www.ebusiness-unibw.org/ontologies/ecla...,C_AKJ315005-tax,1,1
